In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

%load_ext abjadext.ipython
from abjad import *

import numpy as np

from functools import partial, reduce
from itertools import accumulate
from fractions import Fraction
from random import shuffle, random, randint

In [ ]:
def normalize(xs, a=0.0, b=1.0):
    return (b - a) * (xs - np.min(xs)) / (np.max(xs) - np.min(xs)) + a

def compose(*functions):
    return reduce(lambda g, f: lambda x: f(g(x)), functions, lambda x: x)

In [ ]:
def fract(_xs, basis=None):
    if basis is None:
        basis = _xs
    new_xs = [normalize(_xs, basis[0], basis[1])[0]]
    
    for xi, xii in zip (basis[:-1], basis[1:]):
        new_xs += list(normalize(_xs, xi, xii))[1:]
      
    return new_xs

In [ ]:
# xs = [0., 0.33, .75, 1.]
# ys = [-1., .75, -.33, 1.]
xs = [0., 3/12, 7/12, 1.]
ys = [-1., 3/12, -5/12, 1.]

In [ ]:
xs = list(accumulate([random() for _ in range(4)]))
ys = [randint(-6, 6) for _ in range(4)]

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(xs, ys)
plt.show()

In [ ]:
mxs, mys = fract(xs), fract(ys)
# mxs, mys = fract(mxs), fract(mys)
# mxs, mys = fract(mxs), fract(mys)

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(mxs, mys)
plt.show()

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(fract(mxs, xs), fract(mys, ys))
plt.show()

In [ ]:
pitches = normalize(ys, -6., 6.)
durations = list(normalize(np.diff(xs), 0., 3.))  + [1.]
notes = [Note(pitch, Duration((2 ** round(duration), 32))) for (pitch, duration) in zip(pitches, durations)]

notes += [Rest((1, 4))]

pitches = normalize(mys, -12., 12.)
durations = list(normalize(np.diff(mxs), 0., 3.)) + [1.]
notes += [Note(pitch, Duration((2 ** round(duration), 32))) for (pitch, duration) in zip(pitches, durations)]

notes += [Rest((1, 4))]

pitches = normalize(fract(mys, ys), -18., 18.)
durations = list(normalize(np.diff(fract(mxs, xs)), 0., 3.)) + [1.]
notes += [Note(pitch, Duration((2 ** round(duration), 32))) for (pitch, duration) in zip(pitches, durations)]

staff_1 = Staff(notes)
# staff_2 = Staff(notes[:len(notes)//2])

#staff_group = StaffGroup([staff_1, staff_2])
#leaf = inspect(staff_2).get_leaf(0)
#attach(Clef('bass'), leaf)
show(staff_1)

In [ ]:
play(staff_1)